In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd 
from config import AGES, median
from functions import get_age, build_query, parse_float
import os 
import itertools
import numpy as np 
import openpyxl


In [2]:
import pandas
pandas.__version__

'1.5.3'

In [3]:
import gc
import re

In [4]:
from numpy import nan

In [5]:
from datetime import datetime 

today = datetime.today()

In [6]:
### function definition
# --------------------

# function to set variable values
def create_new_variable(row, frame):
    subset = frame[frame.DATAYEAR == row.year]
    res = subset.query(row['conditions'])
    if not res.empty:
        frame.loc[res.index, row.variable] = row.code 
    

      
# function to count how many new_variables should be created
def count_new_variable(row, frame):
    subset = frame[frame.DATAYEAR == row.year]
    index = pd.MultiIndex.from_tuples([(row.variable, row.code)], names=['variable', 'code'])
    columns = pd.MultiIndex.from_tuples([('Frequency', row.year), ('Weighted Frequency', row.year)])
    
    # query results
    qres = subset.query(row['conditions'])
    
    frequency = 0 if qres.empty else qres.shape[0]
    weighted_frequency = 0 if qres.empty else qres.FINALWT.sum()
    
    # output results
    res = pd.DataFrame(index=index, columns=columns, data = [[frequency, weighted_frequency]])
    return res 
        
# explode loop function
def explode_loop(frame):
    cols = list(frame)
    colvals = [frame[col].dropna().drop_duplicates().unique() if col != 'DATAYEAR' else frame[col].dropna().unique() for col in list(frame)]
    res = pd.DataFrame([dict(list(zip(cols, combo))) for combo in itertools.product(*colvals)])
    
    return res

# from a given dataframe, find all the rows in which a given column is not na 
def get_rows_notna(frame, column):
    return frame[frame[column].notna()].index

def update_frame_with_loop_row(frame, column, row):
    # get all the indicies where the given column is not na 
    inds = get_rows_notna(frame, column)
    
    
    frame_update = pd.concat([row.to_frame().T[[column]]]*len(inds))
    frame_update.index = inds 

    frame.update(frame_update)
    
def grab_variable_labels(varname):
    labels = nvdf[nvdf.variable == varname].sort_values('code')[['label', 'code']].drop_duplicates().values
  
    labels =  [tuple(x) for x in labels] + [('', 'Blank')]
    return pd.DataFrame(index = pd.MultiIndex.from_tuples(labels, names=['label', 'code']),
                        columns = pd.MultiIndex.from_tuples([('Frequency', -1), ('Weighted Frequency', -1)])) 
    
def get_count(frame, column):
    try:
        valcount = frame[column].value_counts(dropna=False).sort_index()
    except TypeError as e :
        print(column)
        raise e
    valcount = valcount.to_frame()
    
    valcount['FINALWT'] = valcount.apply(lambda row: frame[frame[column].isin([row.name])].FINALWT.sum(), axis=1)
    valcount.index = valcount.index.fillna('Blank')
    valcount.index.names = ['code']
    
    return valcount
    
def count_present_new_variables(column, labels, frame):
    res = frame.groupby('DATAYEAR').apply(get_count, column)
    res.columns = ['Frequency', 'Weighted Frequency']
    res = res.pivot_table(columns=['DATAYEAR'], index=['code'])
    minyear = frame.DATAYEAR.min()
    maxyear = frame.DATAYEAR.max()
    # the labels column will introduce -1 values to the `weighted and unweighted frequncy` columns
    res = pd.merge(labels, res, left_index=True, right_index=True, how='outer').sort_index(axis=1)
    # removing them using index splicing
    return res.loc[:, pd.IndexSlice[:, minyear:maxyear]]

    
      

In [7]:
def count_from_query(condition, frame):
    if condition and (condition != 'None'):
        res = frame.query(condition)
        
        s = pd.Series(dtype=float)
        if not res.empty:
            s['Frequency'] = res.shape[0]
            s['Weighted Frequency'] = res['FINALWT'].sum()
        else:
            s['Frequency'] = 0
            s['Weighted Frequency'] = 0 
        return s

In [8]:
# loop all of this into a function which you can use for loop.apply(lambda row: ...)
# params so far: row, nvdf (new_variable_frame), analysis_df

# update the coresponding columns new dataframe tables. 

def create_variables_and_perform_analysis(row, df, nvdf, dfs):
    # after you have updated the column values, use the the loop's row to filter out the dataframe?
    filter_str = build_query(**row.to_dict())
    if not nvdf.empty:
        df.drop(columns=new_variables, inplace=True, errors='ignore')
    df1 = df.query(filter_str)


    # create new variables if it's not empty
    if not nvdf.empty:
        df1[new_variables] = np.nan
        # filter the new variables to appropriate year 
        # create year_new_variable_df == ynvdf
        ynvdf = nvdf[nvdf.year == row.DATAYEAR]

        # if the year is not present in the new variable df (nvdf) then skip this year 
        # if ynvdf.empty:
        #     return 

        for rowname in row.index:
            if rowname in ynvdf:
                notnaindex = ynvdf[ynvdf[rowname].notna()].index
                if len(notnaindex) == 0:
                    if rowname == '_AGEG5YR':
                        notnaindex = ynvdf.index
                    else:
                        continue
                naupdate = pd.concat([row.to_frame().T]*len(notnaindex)).reset_index(drop=True)
                
                # ynvdf subset == ts
                ynvdf.loc[notnaindex, rowname] = row.loc[rowname]
        
        ynvdf =  ynvdf.sort_values(['year', 'order', 'variable'])
        # create conditions based on the updates
        ynvdf['conditions'] = ynvdf.apply(lambda row: build_query(**row[variable_cols].dropna().to_dict()),axis=1)
        
        # create new variables based on filtered year data
        ynvdf.apply(lambda row: create_new_variable(row, df1), axis=1)
        
        # if new variable breakdown requested. 

    # lst=[{k:v} for k,v in dfs.items()]
    # lst=lst[0]    
    # apply analysis
    # for name, table in lst.items():
        
    for name, table in dfs.items():
        tb = table.copy(deep=True)
        output_selector = str(row.drop('DATAYEAR', errors='ignore').values.tolist() + [name])
            
        for cols in tb:
            nona =tb[tb[cols].notna()] 
            match = nona[nona[cols].astype(str).str.contains(ptn1)]
            if not match.empty:
                nomatch = match[~match[cols].str.contains(str(row.DATAYEAR))]
                match = match.drop(index = nomatch.index)
                nomatch[cols] = nomatch[cols].replace(ptn1, '', regex=True)
                match[cols] = match[cols].apply(replace_w_match, pattern=ptn3)
                match = pd.concat([match, nomatch])
                tb.loc[match.index] = match
     
        for rowname in row.index:
            if rowname in tb:
                notnaindex = tb[tb[rowname].notna()].index
                if len(notnaindex) == 0:
                    if rowname == '_AGEG5YR':
                        notnaindex = tb.index
                    else:
                        continue
                naupdate = pd.concat([row.to_frame().T]*len(notnaindex)).reset_index(drop=True)
                
                # table subset == ts
                tb.loc[notnaindex, rowname] = row.loc[rowname]
                
        if not output_selector in outputs:
            outputs[output_selector] = tb.drop(columns='conditions', errors='ignore').copy(deep=True)
            outputs[output_selector] = pd.DataFrame()

        varcols = [col for col in list(tb) if col.upper() == col]
        tb['conditions'] = tb.apply(lambda row: build_query(**row[varcols].dropna().to_dict()),axis=1)
        df3 = tb.apply(lambda x: count_from_query(x.conditions, df1), axis=1)
        df3.columns = pd.MultiIndex.from_tuples([('Frequency', row.DATAYEAR), ( 'Weighted Frequency', row.DATAYEAR)])
        
        # print(df3.columns)
        # print(df3.shape)
        # print(outputs[output_selector].shape)
        # print(outputs[output_selector].columns)

        outputs[output_selector] = pd.merge(outputs[output_selector], df3, left_index=True, right_index=True, how='outer').sort_index(axis=1)
         

     # create new variables based on filtered year data
    if not nvdf.empty:
        df1.drop(columns=new_variables, inplace=True, errors='ignore')

In [9]:
def test_create_variables_and_perform_analysis_create_new_variables(row, df, nvdf):
    filter_str = build_query(**row.to_dict())
    if not nvdf.empty:
        df.drop(columns=new_variables, inplace=True, errors='ignore')
    df1 = df.query(filter_str)

    if not nvdf.empty:
        df1[new_variables] = np.nan
        # filter the nvew variables to appropriate year 
        # create year_new_variable_df == nvdf
        update = pd.concat([row.to_frame().T]*len(nvdf)).reset_index(drop=True)
        nvdf.update(update)
        if '_AGEG5YR' in row:
            if row._AGEG5YR in [np.nan]:
                nvdf['_AGEG5YR'] = row._AGEG5YR
  
        
        
        # nvdf =  nvdf.sort_values(['year', 'order', 'variable'])
        # create conditions based on the updates
        nvdf['conditions'] = nvdf.apply(lambda row: build_query(**row[variable_cols].dropna().to_dict()),axis=1)
        
        # create new variables based on filtered year data
        df1.drop(columns=new_variables, inplace=True, errors='ignore')
        df1[new_variables] = np.nan
        nvdf.apply(lambda row: create_new_variable(row, df1), axis=1)
        

    return pd.concat([vnames, vnames.apply(lambda x: count_present_new_variables(x.varname, x.labels, df1), axis=1)], axis=1)

In [10]:
### CONSTANTS
ptn1 = '\(.* in \d{2,4}\)'
ptn2 = '(?<=\s)\d{2,4}'
ptn3 = '(?<=\().*(?=\sin)'
def replace_w_match(string, pattern):
    res = re.search(pattern, string).group()
    if '**' in string :
        res += ' **' 
    return res

In [11]:
## step 1: open files 
# --------------------

xl = pd.ExcelFile(r'C:\Users\AnkitB\OneDrive - crisil.com\backup\Bernstein\Input folder - zip\settings.xlsx')
settings = xl.parse('settings', index_col='setting')
loop = None if settings.loc['LOOP','value'] in [None, False] else xl.parse('loop', na_filter=False).replace('', np.nan)
xl.close()

# analysis excel sheet
axl = pd.ExcelFile(settings.loc['ANALYSIS FILE LOCATION', 'value'])

pages = settings.loc['ANALYSIS PAGE NAMES', 'value']
if isinstance(pages, str):
    pages = pages.split(';')
elif isinstance(pages, (int,float)):
    pages = str(pages)

In [12]:
## step 1.1 : read data required files
# --------------------

# print('========= READING DATA =============')
df = pd.read_csv(settings.loc['DATA LOCATION','value'])
# df.rename(columns={'VCLTEST':'VCLNTEST'},inplace=True)

print('========= READING VARIABLE SET UP FILE =============')
# new variable dataframe will be empty if new variables == false and you'd open it if it isn't
nvdf = pd.DataFrame() if not settings.loc['NEW VARIABLES', 'value'] else pd.read_excel(settings.loc['NEW VARIABLE LOCATION', 'value'], na_filter=False).replace('', np.nan)
outpath = settings.loc['OUTPUT PATH', 'value']

print('========= PARSING DATA ANALYSIS PAGES INTO LOCAL MEMORY =============')
# analysis tables
dfs = {page.strip(): axl.parse(str(page.strip()), skiprows=(settings.loc['ANALYSIS START ROW', 'value'])-1) for page in pages}
axl.close()


========= READING VARIABLE SET UP FILE =============
========= PARSING DATA ANALYSIS PAGES INTO LOCAL MEMORY =============


In [13]:
for ind, col in settings.iterrows():
    print(ind,':', f'"{col.values[0]}"')

NEW VARIABLES : "True"
NEW VARIABLE LOCATION : "C:\Users\AnkitB\OneDrive - crisil.com\backup\Bernstein\Input folder - zip\NEW VARIABLES\colorectal_analysis_new_variables_definition_file.xlsx"
ANALYSIS PAGE NAMES : "1;2;3;4"
ANALYSIS START ROW : "4"
ANALYSIS FILE LOCATION : "C:\Users\AnkitB\OneDrive - crisil.com\backup\Bernstein\Input folder - zip\Colorectal data format UPDATED.xlsx"
LOOP : "True"
OUTPUT PATH : "C:\Users\AnkitB\OneDrive - crisil.com\backup\Bernstein\output_test\new\Colorectal Data Analysis-{0}.xlsx"
DATA LOCATION : "C:\Users\AnkitB\OneDrive - crisil.com\backup\Bernstein\output_test\new\BRFSS_COLORECTAL.csv"


In [14]:
# UNCOMMENT BELOW IF YOU NEED TO REDOWNLOAD THE NEW VARIABLE DATAFRAME
# nvdf = pd.DataFrame() if not settings.loc['NEW VARIABLES', 'value'] else pd.read_excel(settings.loc['NEW VARIABLE LOCATION', 'value'])


# STEPS to follow 
-------------
 step 1:
 open settings file

 step 2: 
 check if you are creating new variables for this analysis 
 if yes, open the location of the settings file and then work on creating new variables

 step 3:
 read the line which tells you which pages you're performing your analysis on (names separated by square brackets
)
 read the tables in each page you're applying your queries to.drop any na rows
 get the list of years you will be applying this to. 
 get list of looping variables you will be applying the same function to

 step 4:
 filter to DATAYEAR variable given in the list of years 
 create conditions for each subset by looping through the rows
 apply conditions and count outputs by creating multi-index for year and frequency / weighted frequency

 step 5:
 prepare way to write to back to openpyxl in a specific order



In [15]:
outputs = {}

In [16]:
l = explode_loop(loop)

In [17]:
l._AGEG5YR.unique()

array(['<45', '45,50', '50,75', '75,80', '>80', '#NA'], dtype=object)

In [18]:
# step 2: check new variables, create if true

if not nvdf.empty:
    # filtering out all the year variable for now
#     nvdf = nvdf[~nvdf.variable.str.startswith('YEAR')].sort_values(['year', 'order'])
    
    # reindex columns to include new variables being crated
    df = df.reindex(list(df)+list(nvdf.variable.unique()),  axis=1)
    variable_cols = [col for col in list(nvdf) if col == col.upper()]
    
    ndf = nvdf.copy(deep=True)
    ndf = ndf.sort_values(by=['year', 'order'])
    
    new_variables = nvdf.variable.unique()
    
    vnames = pd.DataFrame(data=new_variables, columns=['varname'])
    vnames['labels'] = vnames.varname.apply(grab_variable_labels)
# if you have a loop utilise it by exploding it then apply the changes to each row
# if not loop.empty:
#     loop = explode_loop(loop)
    


In [19]:
for _,row in l.sort_values('DATAYEAR').iterrows():
    filter_str = build_query(**row.to_dict())
    if not nvdf.empty:
        df.drop(columns=new_variables, inplace=True, errors='ignore')
    df1 = df.query(filter_str)

    df1[new_variables] = np.nan
    # filter the new variables to appropriate year 
    # create year_new_variable_df == ynvdf
    ynvdf = nvdf[nvdf.year == row.DATAYEAR]   

    # if the year is not present in the new variable df (nvdf) then skip this 
    # if ynvdf.empty:
    #     print('Data year', row.DATAYEAR, 'not found in new variable dataframe')
    #     continue

    for rowname in row.index:
        if rowname in ynvdf:
            notnaindex = ynvdf[ynvdf[rowname].notna()].index
            if len(notnaindex) == 0:
                if rowname == '_AGEG5YR':
                    notnaindex = ynvdf.index
                else:
                    continue

            naupdate = pd.concat([row.to_frame().T]*len(notnaindex)).reset_index(drop=True)
     

            # ynvdf subset == ts
            ynvdf.loc[notnaindex, rowname] = row.loc[rowname]
            
    ynvdf = ynvdf.sort_values(['year', 'order', 'variable'])
    # create conditions based on the updates
    ynvdf['conditions'] = ynvdf.apply(lambda row: build_query(**row[variable_cols].dropna().to_dict()),axis=1)

    # create new variables based on filtered year data
    ynvdf.apply(lambda row: create_new_variable(row, df1), axis=1)
    
    if "#NA" in row.values:
        break 

In [20]:
print(df.DATAYEAR.unique())

[2022]


In [21]:
outputs = {}
l.apply(create_variables_and_perform_analysis, args=(df, nvdf, dfs), axis=1)


0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
dtype: object

In [22]:
import pandas
pandas.__version__

'1.5.3'

In [23]:
loopcols = list(loop.drop(columns='DATAYEAR'))
for i in outputs:
    group = eval(i)
    name = group[-1]
    d = dfs[name].drop(columns='conditions', errors='ignore')
    
    for ind, rowname in enumerate(loopcols):       
        if rowname in d:
            notnaindex = d[d[rowname].notna()].index
            if len(notnaindex) == 0:
                if rowname == '_AGEG5YR':
                    notnaindex = d.index
                else:
                    continue 
            d.loc[notnaindex, rowname] = group[ind]
        else:
            print(f'{rowname} not in d\n', d.columns)
    d.columns = pd.MultiIndex.from_tuples(tuple([('', x) for x in list(d)]))
    
    c = outputs[i]
    c.columns = pd.MultiIndex.from_tuples(tuple([(str(x) for x in x) for x in list(c)]))
    
    outputs[i] = pd.merge(d, c, left_index=True, right_index=True)

In [24]:
data_groups = {}
for page in dfs:
    if not page in data_groups:
        res = [outputs[x] for x in outputs if eval(x)[-1] == page]
        out = pd.DataFrame()
        for x in res:
            out = pd.concat([out, x, pd.DataFrame()])
        out.reset_index(drop=True, inplace=True)
        data_groups[page] = out
        

In [25]:
writer = pd.ExcelWriter(settings.loc['OUTPUT PATH', 'value'].format(str(today.date())))
for k,v in data_groups.items():
    v.to_excel(writer, sheet_name=k)
writer.save()
writer.close()

In [26]:
# settings.loc['OUTPUT PATH', 'value']

In [27]:
# queries = dfs['1'][list(dfs['1'])[3:]].dropna(how='all').apply(lambda x: x.dropna().to_dict(),axis=1)

In [28]:
# queries.apply(lambda x: build_query(**x))

In [29]:
# build_query(**{'EVERANYSCREEN': 1.0, 'UPTODATEANY': 1.0, '_AGEG5YR': '45-49'})

In [30]:
# testing for age group <45,50 

df[df._AGEGROUP]

AttributeError: 'DataFrame' object has no attribute '_AGEGROUP'